# Interactive Tutorial for Hodgkin Huxley Model on Juypter Lab

<font size="5">HH Model for Single Neuron</font>

This model relies on a basic equivalence between a biological membrane plus embedded ion channels, and an electrical circuit.

<font size="5">Basic inputs to the Model</font>

1) Membrane capacitance, $\mu{A}/cm^2$
2) Maximum Conductances, $mS/cm^2$
3) Nernst Reverasal Potentials, $mV$
4) Simulation Parameters (time), $ms$

<img src="equivalentCircuit.PNG" width="500"/>


<font size="5">Execute Hodgkin Huxley Model</font>

In [1]:
import ipywidgets
import ui_widget
from importlib.machinery import SourceFileLoader
import numpy as np
import matplotlib.pyplot as plt
  
# imports the module from the given path
HHmodel = SourceFileLoader("HodgkinHuxley.py","../Tutorial/Source/HodgkinHuxley.py").load_module()

#function to call python script as a module
def runHH(C_m, g_Na, g_K, g_L, E_Na, E_K, E_L, t_0, t_n, delta_t, I_inj_max, I_inj_width, I_inj_trans):
    runner = HHmodel.HodgkinHuxley(C_m, g_Na, g_K, g_L, E_Na, E_K, E_L, t_0, t_n, delta_t, I_inj_max, I_inj_width, I_inj_trans)
    runner.Main()

#function to handle checkbox event
def checkboxEvent(checkboxStatus):
    if checkboxStatus:
        wid_inj=ipywidgets.interact(ui_widget.injectorCurrent,amplidute=ui_widget.slider_amplitude,t_width=ui_widget.slider_width,t_translation=ui_widget.slider_translation);
        display(wid_plotArea)
    else:
        display(wid_plotArea)

#create checkbox widget
wid_checkbox=ipywidgets.interactive(checkboxEvent,checkboxStatus=ui_widget.cb);

#create plot area widget and interact with HHmodel
wid_plotArea=ipywidgets.interactive_output(runHH,{'C_m':ui_widget.slider_capacitance,
                                        'g_Na':ui_widget.slider_cond_Na, 'g_K':ui_widget.slider_cond_K, 'g_L':ui_widget.slider_cond_L, 
                                        'E_Na':ui_widget.slider_pot_Na, 'E_K':ui_widget.slider_pot_K, 'E_L':ui_widget.slider_pot_L,
                                        't_0':ui_widget.time_start, 't_n':ui_widget.time_end, 'delta_t':ui_widget.time_step, 
                                        'I_inj_max':ui_widget.slider_amplitude,'I_inj_width':ui_widget.slider_width,'I_inj_trans':ui_widget.slider_translation})
  
display(ui_widget.basicInputs,wid_checkbox)

interactive(children=(Checkbox(value=False, description="<b><font color='blue'>Advanced Input - Incjection Cur…

<font size="5">Description of the Plots</font>

1) Starting from the bottom, the first (bottom-most) plot shows two currents injected into the cell membrane at times 100ms and 300ms.
2) The second plot from the bottom shows the activation/inactivation parameters of the ion channels in the neuron. 
3) Third plot from the bottom (the current/time plot) makes this more concrete, showing the influx (negative y-axis) and outflux (positive y-axis) of ions passing through each type of ion channel being modeled.
4) The top plot, which shows neural membrane voltage activity. The spikes here are called “action potentials” and correspond directly to the current/time plot. 